In [ ]:
!pip install langchain_openai

In [ ]:
import os

# 換成自己 OPENAI_API_KEY
os.environ["OPENAI_API_KEY"]  = ""

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

# 來函的 chain / agent 

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Optional, List

# 定義分析來函 chain / agent 的 "重點欄位"

class CorrespondenceSchema(BaseModel):
    type: str = Field(description="本別，公文的類別。本別種類有：正本、副本、抄本、影本、譯本等。不同的本別代表文件的正式性和用途，如正本為正式文件，副本通常為參考或備查之用。")
    code: str = Field(description="檔號，文件編號，是用來標識和管理公文的唯一標識碼。通常由發文機關設定，用以便於文件的檢索和查詢。")
    retention_period: str = Field(description="保存年限，指該公文的保存期限。根據公文的重要性和性質，保存年限會有所不同，可能包括永久保存、長期保存、中期保存或短期保存。")
    agency_name: str = Field(description="機關全銜，公文的發文機關全稱，通常為機構的正式名稱。這部分應該使用正式的全銜，避免簡稱。")
    document_type: str = Field(description="文別，公文的類型，根據用途不同可分為函、簽、公告、通報等。文別決定了公文的用途和格式。")
    agency_address: str = Field(description="機關地址，發文機關的地址，需詳細填寫包括郵遞區號在內的完整地址，以便於聯繫。")
    officer_name: str = Field(description="承辦人，負責處理該公文的聯絡人，應該包括其姓名，以便受文者可以聯繫到具體負責人。")
    telephone: str = Field(description="電話，承辦人的聯絡電話，通常應包括區號，並使用半形格式填寫。")
    fax: str = Field(description="傳真，承辦人的傳真號碼，通常用於接收或傳送公文的紙質版。")
    email: str = Field(description="E-mail，承辦人的電子郵件地址，用於電子溝通或公文的電子版傳遞。電子郵件地址應該用半形格式並且遵循正確的格式。")
    recipient_zipcode: str = Field(description="受文者郵遞區號，受文機關或單位的郵遞區號，應使用半形格式填寫。")
    recipient_address: str = Field(description="受文者地址，受文機關或單位的地址，需詳細填寫以確保郵件能準確送達。")
    recipient_name: str = Field(description="受文者，公文的接收者，可以是單位、部門或個人，通常會在此處填寫完整的名稱。")
    dispatch_date: str = Field(description="發文日期，發文的日期，使用中華民國年號格式，通常為「中華民國XXX年XX月XX日」。日期格式應保持一致並使用半形格式。")
    document_number: str = Field(description="發文字號，公文的編號，通常格式為「機關名稱字第XXXX號」，用來標識該公文的身份和序列。")
    priority: str = Field(description="速別，指示處理公文的緊急程度，例如「普通件」、「速件」、「最速件」。這會影響公文的優先處理順序。")
    security_classification: Optional[str] = Field(description="密等及解密條件或保密期限，標識公文的機密等級及其解密條件或保密期限。這一欄通常適用於涉及機密信息的公文。")
    attachments: Optional[str] = Field(description="附件，公文中隨附的其他文件，應列明附件的名稱和數量，以方便核對。")
    subject: str = Field(description="主旨，公文的主要內容摘要，通常簡明扼要地說明公文的目的和要處理的事項。")
    explanation: str = Field(description="說明，對公文的背景、依據和細節進行詳細說明，通常分條列述。")
    methods: Optional[str] = Field(description="辦法，公文中所指示的具體辦理方式和要求，也通常分條列述。")
    original_recipient: Optional[str] = Field(description="正本，發給主要受文者的正式版本，列出其單位名稱。")
    copy_recipient: Optional[str] = Field(description="副本，發給其他相關單位或人員的副版本，列出其單位名稱。")
    signature_or_stamp: Optional[str] = Field(description="署名或蓋章戳，公文發文機關負責人的簽名或機關的正式印章，代表該公文的合法性與正式性。")
    

# 定義分析來函 chain / agent 的 "職責"
    
correspondence_system_prompt = """
    你是台灣公文方面的專家，你了解各式各樣的公文類別以及寫作方式
    這是公文的架構 :

    本別 (標楷體16號字) 檔 號：(標楷體10號字)
    保存年限：(標楷體10號字)
    (機 關 全 銜) (文別) (標楷體20號字)
    機關地址： (標楷體12號字、英數字半形)
    承 辦 人： (標楷體12號字)
    電 話： (標楷體12號字、英數字半形)
    傳 真： (標楷體12號字、英數字半形)
    E-mail ： (標楷體12號字、中文全形、英數字半形)
    (受文者郵遞區號) (標楷體12號字、英數字半形)
    (受文者地址) (標楷體12號字)
    受文者： (標楷體16號字)
    發文日期：中華民國000年00月00日 (標楷體12號字、英數字半形)
    發文字號：○○○字第00000000000號(標楷體12號字、中文全形、英數字半形)
    速 別： (標楷體12號字、中文全形)
    密等及解密條件或保密期限： (標楷體12號字、中文全形、英數字半形)
    附件： (標楷體12號字、中文全形、英數字半形)
    主旨： (標楷體16號字、中文全形、英數字半形)
    說明： (標楷體16號字、中文全形、英數字半形)
    一、 (標楷體16號字、中文全形、英數字半形)
    二、 (標楷體16號字、中文全形、英數字半形)
    辦法： (標楷體16號字、中文全形、英數字半形)
    一、 (標楷體16號字、中文全形、英數字半形)
    二、 (標楷體16號字、中文全形、英數字半形)
    正本：(標楷體12號字)
    副本：(標楷體12號字)
    署名(標楷體16號字)或蓋章戳

    你現在的任務就是將這個架構裡會變動的欄位抽取出來成json 格式，像是 
    \n{format_instructions}，你在抽取的過程中，發現值不應該是該欄位，那就給空字串。\n
    這是原始文檔
    {correspondence_content}\n
"""

In [ ]:
# 定義 LLM模型
correspondence_extract_llm = ChatOpenAI(model="gpt-4o-mini",metadata={"name":"correspondence_extract_llm"})

# 定義 chain 的 output
correspondence_parser = JsonOutputParser(pydantic_object=CorrespondenceSchema)

# 定義 chain 的 prompt
correspondence_prompt = PromptTemplate(
    template=correspondence_system_prompt,
    input_variables=["correspondence_content"],
    partial_variables={"format_instructions": correspondence_parser.get_format_instructions()},
)

# 定義分析來函的 chain 
correspondence_extract_chain = correspondence_prompt | correspondence_extract_llm | correspondence_parser

# 嘗試解析看看來函

In [ ]:
correspondence_content = """
    檔號：1W1120006623
    保存年限：永久
    台中市政府 函
    地址：330206 台中市台中區縣府路 1 號
    承辦人：專案管理員 張子庭
    電話：03-3365076#19
    傳真：03-4092919
    電子信箱：80028699@mail.tycg.gov.tw
    受文者：台中市某某區公所
    發文日期：中華民國 112 年 9 月 6 日
    發文字號：府社區字第 112030251 號
    速別：普通件
    密等及解密條件或保密期限：無
    附件：如主旨 (里民健康日-會議記錄.pdf, 里民健康日-出席名單.pdf)
    主旨：檢送 112 年 9 月 1 日召開「社區健康促進日活動籌備會議」會議紀錄及出
    席名單各 1 份，請查照並賡續辦理相關事宜。
    說明：一、依本府推動社區健康促進活動政策，並配合 112 年 10 月 15 日舉辦的「社區
    健康促進日」活動，特召開籌備會議。
    二、請貴單位依會議決議內容，協助辦理宣傳、場地確認及物資準備等相關事宜。
    三、另請於活動後進行成效評估，並將報告結果呈報本府。
    正本：台中市某某區公所
    副本：社區健康促進協會、某某區合作醫院、營養健康中心、心理諮商中心
    規劃發展科
    112/09/06
"""

In [ ]:
correspondence_extract_result = correspondence_extract_chain.invoke({'correspondence_content':correspondence_content})

In [ ]:
correspondence_extract_result

# 分析會議記錄 (附檔)

In [ ]:
# 定義常見的會議紀錄的 "重點欄位"

class MeetingMinutesSchema(BaseModel):
    title: str = Field(description="會議標題，或是會議的目的")
    meeting_time: str = Field(description="開會時間，記錄會議開始的日期和時間")
    location: Optional[str] = Field(description="會議地點，記錄會議舉行的場所")
    chairperson: Optional[str] = Field(description="主席，主持會議的負責人")
    recorder: Optional[str] = Field(description="紀錄人，負責記錄會議內容的成員")
    attendees: Optional[List[str]] = Field(description="出席人員，列出參加會議的所有成員")
    chairperson_remarks: str = Field(description="主席致詞，記錄會議開始時主席的致詞或開場白")
    discussion_points: Optional[str] = Field(description="會議討論重點，或是報告事項，記錄會議中討論的重要議題或報告的內容")
    resolutions: Optional[str] = Field(description="決議事項，記錄會議中通過的決議或所做出的決定")
    motions: Optional[str] = Field(description="臨時動議，記錄會議中的臨時提案或討論事項")
    adjournment_time: Optional[str] = Field(description="散會時間，記錄會議結束的時間")
    

# 定義分析會議紀錄 chain / agent 的 "職責"

meeting_minutes_system_prompt = """
    你是會議記錄撰寫的專家，你了解各式各樣的會議記錄格式以及寫作方式
    這是會議記錄的架構：
    
    會議標題
    開會時間
    會議地點
    主席
    紀錄人
    出席人員
    主席致詞
    會議討論重點 或是 報告事項
    決議事項
    臨時動議
    散會時間
    
    在會議記錄中，一些欄位可能不會出現在文本中，例如會議地點、紀錄人、出席人員、決議事項、臨時動議和散會時間。你的任務是根據這個架構，將文本中出現的對應欄位提取出來，並轉換為JSON格式。對於那些未出現或沒有相關信息的欄位，請將它們留空（使用空字串表示）。
    
    \n{format_instructions}，你在抽取的過程中，發現值不應該是該欄位，那就給空字串。\n
    這是原始會議記錄：
    {meeting_minutes_content}\n
"""


In [ ]:
# 定義 LLM模型
meeting_minutes_extract_llm = ChatOpenAI(model="gpt-4o-mini",metadata={"name":"meeting_minutes_extract_llm"})

# 定義 chain 的 output
meeting_minutes_parser = JsonOutputParser(pydantic_object=MeetingMinutesSchema)

# 定義 chain 的 prompt
meeting_minutes_prompt = PromptTemplate(
    template=meeting_minutes_system_prompt,
    input_variables=["meeting_minutes_content"],
    partial_variables={"format_instructions": meeting_minutes_parser.get_format_instructions()},
)

# 定義分析來函的 chain 
meeting_minutes_extract_chain = meeting_minutes_prompt | meeting_minutes_extract_llm | meeting_minutes_parser

In [ ]:
meeting_minutes_content_list = [
    """
        社區健康促進日活動籌備會議紀錄
        壹、時間：112 年 9 月 1 日（星期五）上午 10 時
        貳、地點：台中市某某區里民活動中心
        參、主席：社區發展科長 李明哲 紀錄：侯專員
        肆、出（列）席人員：詳如出席名單。
        伍、主席致詞：略。
        陸、會議決議：
        一、活動時間確定為 112 年 10 月 15 日（星期日）上午 9 時至下午 5 時，
        活動場地為里民活動中心，並包含戶外廣場。
        二、健康講座主題與講師已確認，講座時間安排為上午 9 時至 10 時，由健
        康促進協會推薦專家擔任主講，內容涵蓋高齡健康管理與慢性疾病預防。
        三、健康檢查區域設置於活動中心內側，合作醫院將提供基本身體檢查
        （血壓、血糖、心電圖等），並安排專業醫護人員進行健康諮詢，現場備有
        流動醫療設備，確保流程順暢。
        四、體驗活動將分為三個區域：
        （一）運動體驗區：由運動科學中心專業指導，設置簡易體適能測試
        與健康運動指導。
        （二）飲食健康區：提供健康飲食示範，並由營養師講解平衡飲食的
        重要性及如何設計個人化健康餐單。
        （三）心理健康區：邀請心理諮商師進行壓力釋放與情緒管理的工作
        坊，並提供現場個別諮詢服務。
        五、宣傳部分：
        （一）請區公所協助張貼活動海報於各里辦公室，並於活動前兩週在
        社區公告欄及網站發布活動訊息。
        （二）社區健康促進協會將於社群媒體與地方報紙上發布相關宣傳資
        料，吸引更多里民參加，並透過社區廣播於活動當日提醒里民活動開
        始時間。
        六、場地佈置與安全措施：
        （一）場地佈置將於活動前一天完成，請區公所協助相關硬體設施的
        準備，包括音響、帳篷、桌椅等，並確保活動當日所有設施運作正
        常。
        （二）消防安全與醫療救護措施已安排，並於活動當日設置緊急醫療
        站及消防通道，保證活動進行中的安全。
        七、物資準備：社區發展科將負責提供活動紀念品，如環保袋、健康小冊
        子等，並安排志工於當日協助物資發放及場內引導。
        八、請社區健康促進協會於活動後進行成效評估，並撰寫報告，內容包括
        參與人數、里民回饋及活動成效分析，以作為後續活動參考。
        九、活動經費由區公所預算支出，協會部分由企業贊助，並安排於活動後
        進行財務結算。
        柒、臨時動議：無。
        捌、散會：是日上午 11 時 45 分。
    """,
    """
        社區健康促進日活動籌備會議
        出席名單
        一、時間：112 年 9 月 1 日（星期五）上午 10 時
        二、地點：台中市某某區里民活動中心
        三、主持人：社區發展科長 李明哲
        四、出席單位及人員：
        單位 姓名
        台中市政府社區發展科 李科長明哲
        台中市某某區公所 王主任佳恩
        社區健康促進協會 張專案經理育瑄
        某某區合作醫院 陳護理長安琪
        營養健康中心 林營養師淑芳
        心理諮商中心 蔡諮商師文婷
        社區發展協會 吳理事長建宏
        志工團隊 劉志工隊長瑞明
    """
]

```
會議紀錄有可能會有多個，故我們將資訊先全部整合起來 

meeting_0:

title : xx
meeting_time : xx
...

########################################################

meeting_1:


title : xx
meeting_time : xx
...
########################################################
```

In [ ]:
all_meeting_minutes_extract_results = ""
for m_idx, meeting_minutes_content in enumerate(meeting_minutes_content_list):
    meeting_minutes_extract_result = meeting_minutes_extract_chain.invoke({'meeting_minutes_content':meeting_minutes_content})
    
    content = ""
    for k, v in meeting_minutes_extract_result.items():
        content += f"{k} : {v}\n"
    
    all_meeting_minutes_extract_results += f"meeting_{m_idx}:\n\n" + content + "\n\n########################################################\n\n"

In [ ]:
print(all_meeting_minutes_extract_results)

# 寫作公文(簽)的 chain / agent 

In [ ]:
document_writing_llm = ChatOpenAI(model="gpt-4o-mini",metadata={"name":"document_writing_llm"})

### 大簽

In [ ]:
# 定義寫大簽的 "職責"以及希望他遵循的"寫作架構"
formal_memo_system_prompt = """
    想像你是負責寫大簽的公文，你會接收到來自外部機構來函以及會議紀錄(有可能是多個紀錄)
    根據來函的基本資訊，以及會議記錄們
    
    來分析並產生一個撰寫大簽的大綱架構，架構如下:
    
    1. 主旨
    格式：起首語 + ○○一案 + 期望語
    內容：以50-60字為原則，扼要說明簽的目的及擬辦，應具有原則性、概括性或方向性。
    2. 說明
    格式：分項敘述
    內容：具體說明案情來源、經過、有關法規或前案及處理方法分析。
    3. 擬辦
    格式：具體意見
    內容：提出具體處理方案或可行作法，應包含細節性意見。
    期望語：像是敬呈、謹陳、敬、陳、此致等等。
    
    下面是範例 :
    
    主旨：有關教育部檢送112年11月28日「數位教育發展會議」會議紀錄一案，請鑒核。
    說明：
    一、依教育部112年12月10日教數發字第1120100321號函辦理。
    二、本次會議結論簡述如下：
    (一)推動數位教育普及化，提升學生數位素養。
    (二)教育部將持續支持學校導入數位學習技術，促進教育公平。
    三、本案暫無本局辦理事項，惟本次會議涉及數位教育議題，由教育推廣科派員與會，會辦教育推廣科知悉。
    擬辦：文擬陳閱後存查。
    
    切記，只要輸出內容即可，不要輸出其他的資訊。
"""

In [ ]:
formal_memo_messages = [
    ("system", formal_memo_system_prompt),
    ("system", """
    會議紀錄有可能會有多個，所以整合起來是 
    meeting_0:
    
    title : xx
    meeting_time : xx
    ...
    
    ########################################################
    
    meeting_1:
    
    
    title : xx
    meeting_time : xx
    ...
    ########################################################
    """
    ),
     ("human", "{correspondence_meta}"),
     ("human", "{all_meeting_minutes_meta}"),
     ("human", "請根據外部機構來函的資訊： {correspondence_meta}"),
     ("human", "以及外部機構來函的會議紀錄： {all_meeting_minutes_meta}"),
     ("human", "我想要撰寫公文的大簽。你能將這些資訊制定一個清晰的大綱架構嗎？")
]

formal_memo_prompt = ChatPromptTemplate.from_messages(formal_memo_messages)
formal_memo_writing_chain = formal_memo_prompt | document_writing_llm | StrOutputParser()

### 把剛剛預先分析的來函重點跟會議記錄重點餵給寫作大簽 chain

In [ ]:
formal_memo_result = formal_memo_writing_chain.invoke({
    'correspondence_meta': correspondence_extract_result,
    "all_meeting_minutes_meta": all_meeting_minutes_extract_results,
})
print(formal_memo_result)

### 小簽/ 便簽

In [ ]:
# 定義寫便簽的 "職責"以及希望他遵循的"寫作架構"
brief_memo_system_prompt = """
    想像你是負責寫便簽的公文，你會接收到來自外部機構來函以及會議紀錄(有可能是多個紀錄)
    根據來函的基本資訊，以及會議記錄們
    
    來分析並產生一個撰寫便簽的大綱架構，架構如下:
    
    一、引據 (寫本案之主要依據，來文機關及文號‧手諭（奉交下）‧會議提示‧業務需要‧事實‧計畫‧方案‧報導)
    二、申述 (寫本案之來龍去脈即事實、原因、理由，陳述現況及問題或分析利弊因素)
    三、歸結 (寫上述原因所造成結果，承辦單位見解)
    四、擬辦意見 (向受文者提出具體要求或作法)
    期望語：像是敬呈、謹陳、敬、陳、此致等等。
    
    下面是範例 :
    
    一、案係xxx年xx月xx日環保署第xxxx次會議中有關「環境監控─強化空氣品質及水質監測計畫」報告案之決定，
    其報告內容主要係為加強空氣品質及水質的監測，導入先進監測技術以提升環境品質(詳參考資料1、2)，
    並規劃未來3年(2024-2026年)共計投入7億元，推動環境監控計畫，以保障公眾健康。 
    二、查本府環境監控及監測技術涉及本局環保科業務，會辦監測科知悉。
    三、文陳閱後存查。
    
    切記，只要輸出內容即可，不要輸出其他的資訊。
"""

brief_memo_messages = [
    ("system", brief_memo_system_prompt),
    ("system", """
    會議紀錄有可能會有多個，所以整合起來是 
    meeting_0:
    
    title : xx
    meeting_time : xx
    ...
    
    ########################################################
    
    meeting_1:
    
    
    title : xx
    meeting_time : xx
    ...
    ########################################################
    """
    ),
     ("human", "{correspondence_meta}"),
     ("human", "{all_meeting_minutes_meta}"),
     ("human", "請根據外部機構來函的資訊： {correspondence_meta}"),
     ("human", "以及外部機構來函的會議紀錄： {all_meeting_minutes_meta}"),
     ("human", "我想要撰寫公文的便簽。你能將這些資訊制定一個清晰的大綱架構嗎？")
]

brief_memo_prompt = ChatPromptTemplate.from_messages(brief_memo_messages)
brief_memo_writing_chain = brief_memo_prompt | document_writing_llm | StrOutputParser()

### 把剛剛預先分析的來函重點跟會議記錄重點餵給寫作大簽 chain

In [ ]:
brief_memo_result = brief_memo_writing_chain.invoke({
    'correspondence_meta': correspondence_extract_result,
    "all_meeting_minutes_meta": all_meeting_minutes_extract_results,
})
print(brief_memo_result)

# 是不是還有能改進的地方?
```
Data Manager
公文用詞
寫作風格
...
```

# 延伸
```
人事時地物的知識圖譜
派工系統
```